In [4]:
from elasticsearch import Elasticsearch, helpers
from math import radians, cos, sin, sqrt, atan2, degrees
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Elasticsearch 연결 설정
es = Elasticsearch("http://localhost:9200")

# 기존 인덱스와 새 인덱스 이름
source_index = "choso"
destination_index = "choso_ladius"

# 새 인덱스 매핑 설정 (geo_shape 필드)
mapping = {
    "mappings": {
        "properties": {
            "location": {
                "type": "geo_shape"
            }
        }
    }
}

# 새 인덱스 생성
if not es.indices.exists(index=destination_index):
  es.indices.create(index=destination_index, body=mapping)
  print(f'{destination_index} :: 생성 완료')
else : 
  # 이미 인덱스가 존재하면 삭제 후 생성
  es.indices.delete(index=destination_index)
  es.indices.create(index=destination_index, body=mapping)
  print(f'{destination_index} :: 인덱스 삭제 후 생성 완료')

# 반경을 기반으로 다각형 생성 (반경: 1km)
def create_circle_as_polygon(lon, lat, radius_km, num_sides=64):
    """
    주어진 반경과 중심점으로부터 반경을 표현하는 다각형 생성.
    :param lon: 중심점 경도
    :param lat: 중심점 위도
    :param radius_km: 반경 (킬로미터)
    :param num_sides: 다각형의 변의 수
    :return: 다각형 좌표 리스트
    """
    earth_radius_km = 6371.0  # 지구 반경 (킬로미터)
    angle_step = 360 / num_sides
    points = []

    for i in range(num_sides):
        angle = radians(i * angle_step)
        dx = radius_km * cos(angle)
        dy = radius_km * sin(angle)

        # 경도/위도 계산
        delta_lon = degrees(dx / (earth_radius_km * cos(radians(lat))))
        delta_lat = degrees(dy / earth_radius_km)

        points.append([lon + delta_lon, lat + delta_lat])

    # 닫힌 다각형 형식으로 반환
    points.append(points[0])
    return points

# 데이터 조회 및 변환
query = {"query": {"match_all": {}}}
scroll = es.search(index=source_index, body=query, scroll="2m", size=1000)
scroll_id = scroll["_scroll_id"]
documents = scroll["hits"]["hits"]

while documents:
    actions = []
    for doc in documents:
        try:
            # 원래 좌표 가져오기
            coordinates = doc["_source"]["location"]
            location = str.split(coordinates, ',')
            latitude = float(location[0])
            longitude = float(location[1])

            # 다각형 생성
            polygon = create_circle_as_polygon(longitude, latitude, radius_km=5)

            # 새로운 문서 생성
            new_doc = {
                "_index": destination_index,
                "_source": {
                    "location": {
                        "type": "polygon",
                        "coordinates": [polygon]
                    }
                }
            }
            actions.append(new_doc)
        except Exception as e:
            print(f"Error processing document ID {doc['_id']}: {e}")

    # 새로운 데이터 저장
    if actions:
        helpers.bulk(es, actions)

    # 다음 batch 로드
    scroll = es.scroll(scroll_id=scroll_id, scroll="2m")
    scroll_id = scroll["_scroll_id"]
    documents = scroll["hits"]["hits"]

# 완료 메시지
print(f"{destination_index}로 데이터 전환 완료")


choso_ladius로 데이터 전환 완료
